In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Getting helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-26 16:47:14--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-26 16:47:14 (58.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-26 16:47:24--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 173.194.203.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-06-26 16:47:24 (61.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing dataset

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
print(f"Total training samples: {len(train_df)}")
print(f"Total testing samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total testing samples: 3263
Total samples: 10876


In [11]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [12]:
import random

In [13]:
random_index = random.randint(0, len(train_df)-5)

for index in range(random_index, random_index+5):
    print(f'Target: {train_df["target"][index]}')
    print(f'Text:\n{train_df["text"][index]}')
    print("\n")

Target: 1
Text:
@tiggr_ why only Squad Obliteration?


Target: 0
Text:
Why did God order obliteration of ancient Canaanites? http://t.co/NLk1DYD2tP


Target: 0
Text:
Back in 02 to 03 would never said that 50 would have ended ja like obliteration


Target: 0
Text:
@Eganator2000 There aren't many Obliteration servers but I always like to play when there are :D


Target: 0
Text:
What a win by Kerry. 7-16..... #obliteration




### Spliting Data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [16]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
temp = pd.DataFrame(data=train_sentences[:10], columns=["text"])
temp["target"] = train_labels[:10]
temp

,text,target
0,@mogacola @zamtriossu i screamed after hitting...,0
1,Imagine getting flattened by Kurt Zouma,0
2,@Gurmeetramrahim #MSGDoing111WelfareWorks Gree...,1
3,@shakjn @C7 @Magnums im shaking in fear he's g...,0
4,Somehow find you and I collide http://t.co/Ee8...,0
5,@EvaHanderek @MarleyKnysh great times until th...,1
6,destroy the free fandom honestly,1
7,Weapons stolen from National Guard Armory in N...,0
8,@wfaaweather Pete when will the heat wave pass...,1
9,Patient-reported outcomes in long-term survivo...,1


### Text vectorization (tokenization)

In [18]:
from tensorflow.keras.layers import TextVectorization

In [19]:
avg_sentence_length = round(sum([len(sentence.split()) for sentence in train_sentences]) / len(train_sentences))
avg_sentence_length

15

In [20]:
# this method's every parameter is set by default, what we need. Just need to change below two params.
text_vectorizer = TextVectorization(max_tokens=10000,
                                    output_sequence_length=avg_sentence_length)

In [21]:
text_vectorizer.adapt(train_sentences)

In [22]:
import random

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Before tokenization: {random_sentence}")
print(f"After tokenization: {text_vectorizer([random_sentence])}")

Before tokenization: This guy idk just made me his woman crush ?? first one ever ??
After tokenization: [[  19  848 2934   29  299   31   79  410  438   97   61  219    0    0
     0]]


In [24]:
print(f"5 most common words: {text_vectorizer.get_vocabulary()[:5]}")
print(f"5 least common words: {text_vectorizer.get_vocabulary()[-5:]}")

5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Embedding

In [25]:
from tensorflow.keras.layers import Embedding

In [26]:
embedding = Embedding(input_dim=len(text_vectorizer.get_vocabulary()),
                      output_dim=128,
                      input_length=avg_sentence_length)

In [27]:
random_sentence = random.choice(train_sentences)
print(f"Sentence before embedding: {random_sentence}")
embedding(text_vectorizer([random_sentence]))

Sentence before embedding: #Horrible #Accident Man Died In Wings Airplane (29-07-2015) #WatchTheVideo http://t.co/p64xRVgJIk


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02329786,  0.03106124,  0.04728854, ...,  0.02448876,
         -0.0083737 ,  0.00949639],
        [-0.02694477,  0.01332866, -0.03950665, ...,  0.03082366,
         -0.01851858, -0.00052929],
        [-0.03997724, -0.0148444 ,  0.01884506, ..., -0.04345858,
          0.01261247, -0.00945116],
        ...,
        [-0.00635873,  0.03652134,  0.03108003, ...,  0.04666207,
          0.01172829,  0.01259755],
        [-0.00635873,  0.03652134,  0.03108003, ...,  0.04666207,
          0.01172829,  0.01259755],
        [-0.00635873,  0.03652134,  0.03108003, ...,  0.04666207,
          0.01172829,  0.01259755]]], dtype=float32)>

**Note:** If we see the shape of the embedded sentence, we will notice it is (1, 15, 128)<br>
which means from every sentence we will take 15 words and each word will be represented with 128 shape matrix.

### `model_0`: Baseline

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [29]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("multinomial", MultinomialNB())
])

In [30]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('multinomial', MultinomialNB())])

### Evaluation function

In [31]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [32]:
def calculate_results(y_true, y_pred):
    """
    returns a dictionary of accuracy_score, precission, recall and f1_score
    """
    precission, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    acc_score = accuracy_score(y_true, y_pred) * 100
    
    results = {
        "accuracy_score": acc_score,
        "precission": precission,
        "recall": recall,
        "f1_score": f1_score
    }
    return results

In [33]:
baseline_results = calculate_results(val_labels, model_0.predict(val_sentences))
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_1`: Simple dense model

In [34]:
import tensorflow as tf
from tensorflow.keras import layers

In [35]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
SAVE_DIR = "model_logs"

In [39]:
history_1 = model_1.fit(train_sentences, 
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                               experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230626-164730
Epoch 1/5
215/215 [==============================] - 18s 63ms/step - loss: 0.6346 - accuracy: 0.6487 - val_loss: 0.5720 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4711 - accuracy: 0.8224 - val_loss: 0.4792 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3447 - accuracy: 0.8739 - val_loss: 0.4534 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2607 - accuracy: 0.9078 - val_loss: 0.4586 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2004 - accuracy: 0.9352 - val_loss: 0.4688 - val_accuracy: 0.7808


In [40]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.37769365],
       [0.83483547],
       [0.9891234 ],
       [0.04943281],
       [0.2448615 ],
       [0.9537086 ],
       [0.93356985],
       [0.98719704],
       [0.9492834 ],
       [0.07079127]], dtype=float32)

In [41]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
model_1_result = calculate_results(y_true=val_labels,
                                 y_pred=model_1_preds)
model_1_result

{'accuracy_score': 78.08398950131233,
 'precission': 0.7841274438015813,
 'recall': 0.7808398950131233,
 'f1_score': 0.7782630349987989}

### `model_2`: LSTM

In [43]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [44]:
model_2.compile(loss="binary_crossentropy", 
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [45]:
history_2 = model_2.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                              "LSTM_model")])

Saving TensorBoard log files to: model_logs/LSTM_model/20230626-164758
Epoch 1/5
215/215 [==============================] - 17s 56ms/step - loss: 0.3743 - accuracy: 0.8365 - val_loss: 0.4900 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2422 - accuracy: 0.9029 - val_loss: 0.4911 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1742 - accuracy: 0.9317 - val_loss: 0.5799 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1327 - accuracy: 0.9504 - val_loss: 0.8169 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0912 - accuracy: 0.9637 - val_loss: 0.9704 - val_accuracy: 0.7598


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10] 

24/24 [==============================] - 1s 3ms/step


array([[9.3408799e-01],
       [8.6086053e-01],
       [9.9898475e-01],
       [1.5030612e-01],
       [2.8168876e-04],
       [9.9947888e-01],
       [7.8186649e-01],
       [9.9958676e-01],
       [9.9961603e-01],
       [5.9597695e-01]], dtype=float32)

In [47]:
model_2_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_result = calculate_results(y_true=val_labels,
                                  y_pred=model_2_pred)
model_2_result

{'accuracy_score': 78.08398950131233,
 'precission': 0.7841274438015813,
 'recall': 0.7808398950131233,
 'f1_score': 0.7782630349987989}

In [49]:
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_3': GRU

In [50]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x)
# print(x.shape)
x = layers.GRU(units=64)(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU_model")

In [51]:
model_3.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [52]:
history_3 = model_3.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "GRU_model")])

Saving TensorBoard log files to: model_logs/GRU_model/20230626-164828
Epoch 1/5
215/215 [==============================] - 16s 55ms/step - loss: 0.1932 - accuracy: 0.9279 - val_loss: 0.6293 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0995 - accuracy: 0.9653 - val_loss: 0.7904 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0797 - accuracy: 0.9702 - val_loss: 1.0158 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0610 - accuracy: 0.9737 - val_loss: 0.9611 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0541 - accuracy: 0.9756 - val_loss: 1.2259 - val_accuracy: 0.7638


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:5]

24/24 [==============================] - 1s 3ms/step


array([[2.7382066e-02],
       [7.8694290e-01],
       [9.9990416e-01],
       [2.1040176e-01],
       [1.1779143e-04]], dtype=float32)

In [54]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [55]:
model_3_result = calculate_results(y_true=val_labels,
                                  y_pred=model_3_pred)
model_3_result

{'accuracy_score': 76.37795275590551,
 'precission': 0.76369847151459,
 'recall': 0.7637795275590551,
 'f1_score': 0.7626922180365151}

### `model_4`: Bidirectional

In [56]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)
# x = layers.Bidirectional(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="bidirectional_model")

(None, 15, 128)
(None, 128)
(None, 128)


In [57]:
model_4.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [58]:
history_4 = model_4.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "bidirectional_model")])

Saving TensorBoard log files to: model_logs/bidirectional_model/20230626-164914
Epoch 1/5
215/215 [==============================] - 20s 59ms/step - loss: 0.1228 - accuracy: 0.9501 - val_loss: 1.0574 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0565 - accuracy: 0.9771 - val_loss: 1.0829 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0486 - accuracy: 0.9778 - val_loss: 1.3233 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0462 - accuracy: 0.9794 - val_loss: 1.3667 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0470 - accuracy: 0.9783 - val_loss: 1.4160 - val_accuracy: 0.7572


In [59]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:5]

24/24 [==============================] - 1s 5ms/step


array([[2.2477571e-04],
       [7.2076023e-01],
       [9.9990582e-01],
       [1.8164466e-01],
       [5.4378124e-05]], dtype=float32)

In [60]:
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [61]:
model_4_result = calculate_results(val_labels, model_4_pred)
model_4_result

{'accuracy_score': 75.7217847769029,
 'precission': 0.7613930485354129,
 'recall': 0.7572178477690289,
 'f1_score': 0.7535490768810932}

### `model_5`: Convolutional network

In [70]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
print(x.shape)
x = layers.GlobalMaxPool1D()(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

(None, 15, 128)
(None, 11, 32)
(None, 32)


In [76]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 1)              

In [71]:
model_5.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [72]:
history_5 = model_5.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230626-170559
Epoch 1/5
215/215 [==============================] - 16s 54ms/step - loss: 0.1694 - accuracy: 0.9508 - val_loss: 0.7878 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0806 - accuracy: 0.9718 - val_loss: 0.8882 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0609 - accuracy: 0.9790 - val_loss: 1.0071 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0546 - accuracy: 0.9793 - val_loss: 1.0248 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0507 - accuracy: 0.9791 - val_loss: 1.1144 - val_accuracy: 0.7388


In [73]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5]

24/24 [==============================] - 0s 2ms/step


array([[5.0185132e-01],
       [9.3130088e-01],
       [9.9998546e-01],
       [8.4814258e-02],
       [4.8889269e-06]], dtype=float32)

In [74]:
model_5_pred = tf.squeeze(tf.round(model_5_pred_probs))
model_5_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 1., 1., 0., 0.], dtype=float32)>

In [75]:
model_5_result = calculate_results(val_labels, model_5_pred)
model_5_result

{'accuracy_score': 73.88451443569554,
 'precission': 0.7383784593023884,
 'recall': 0.7388451443569554,
 'f1_score': 0.7378071190188606}